# 라이브러리 import

In [103]:
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time
import re
from tqdm import tqdm

# 함수 정의

In [104]:
def get_url_list():
    with open("../data/job_link.csv", encoding='cp949') as f:
        urls = f.readlines()

    url_list = []

    for url in urls:
        url_spl = url.rstrip("\n").split(",")

        job = ",".join(url_spl[:-1]).strip('""')
        url = url_spl[-1]

        url_list.append([job, url])
    
    return url_list

def get_job_description(driver, url_list):
    job = url_list[0]
    url = url_list[1]
    
    response = requests.get(url)
    
    if response.status_code == 200:
        # url 접속하기
        driver.get(url)

        # 화면 비율을 축소해서 모든 데이터 보이게 만들기
        driver.execute_script("document.body.style.zoom='20%'")

        time.sleep(5)

        # Job Description 을 담을 딕셔너리 생성
        res = {}

        # url 에서 index 값 추출
        res['id'] = url.split("/")[-1]

        # 직무 이름 값 입력
        res['job'] = job

        # JobHeader 클래스 가져와서 title, company, tags 추출
        job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
        title = job_header.find_element(By.TAG_NAME, 'h2')
        company = job_header.find_element(By.TAG_NAME, 'a')
        location = job_header.find_element(By.CLASS_NAME, 'JobHeader_pcLocationContainer__xRwIv')
        tags = job_header.find_element(By.CLASS_NAME, 'Tags_tagsClass__mvehZ')

        # res 딕셔너리에 저장
        res['title'] = title.text
        res['company'] = company.text
        res['location'] = location.text.split(".")

        # Tag 리스트 가져와서 저장
        tag_list = list(map(str, tags.text.split('#')))
        tag_list = tag_list[1:]
        res['tags'] = tag_list

        # JobDescription 클래스 가져와서 header, contents 추출
        jd = driver.find_element(By.CLASS_NAME, 'JobDescription_JobDescription__VWfcb')
        jd_header = jd.find_elements(By.TAG_NAME, 'h6')
        jd_contents = jd.find_elements(By.TAG_NAME, 'p')

        for i in range(1, len(jd_header)+1):
            title = jd_header[i-1].text
            content = jd_contents[i].text.replace("- ", "").split("\n")

            if title == '자격요건':
                res['qualifications'] = content
            elif title == '우대사항':
                res['preferred'] = content
            elif title == '혜택 및 복지':
                res["benefits"] = content
            elif title == '기술스택 ・ 툴':
                res["skills"] = content


        jwp = driver.find_element(By.CLASS_NAME, "JobWorkPlace_className__ra6rp")
        jwp_bodies = jwp.find_elements(By.CLASS_NAME, 'body')
        workplace = jwp_bodies[1].text
        res['workplace'] = workplace
    else:
        print("존재하지 않는 페이지 입니다.")
    
    return res

In [68]:
# driver = webdriver.Chrome()

# url = ["백엔드 개발자", "https://www.wanted.co.kr/wd/160122"]

# # url 접속하기
# driver.get(url[1])

# # 화면 비율을 축소해서 모든 데이터 보이게 만들기
# driver.execute_script("document.body.style.zoom='20%'")

# time.sleep(5)

# job_header = driver.find_element(By.CLASS_NAME, 'JobHeader_className__HttDA')
# location = job_header.find_element(By.CLASS_NAME, 'JobHeader_pcLocationContainer__xRwIv')
# print(location.text.split("."))

# driver.close()

# 크롤링 실행

In [105]:
driver = webdriver.Chrome()

# url 가져오기
url_list = get_url_list()

total = []

for i in tqdm(range(0, 500)):
    res = get_job_description(driver, url_list[i])
    total.append(res)

driver.close()

  0%|▏                                                                                 | 1/500 [00:08<1:06:59,  8.06s/it]

1 웹 서버/백엔드(Back-End) 개발자


  0%|▎                                                                                   | 2/500 [00:14<58:48,  7.09s/it]

2 Backend 개발팀 시니어 개발자 (6년 이상)


  1%|▌                                                                                   | 3/500 [00:20<56:20,  6.80s/it]

...

4 React 개발자


  1%|▊                                                                                   | 5/500 [00:33<54:44,  6.63s/it]

5 [firebase,Vue.js] 풀스택 개발자


  1%|▊                                                                                 | 5/500 [00:36<1:00:56,  7.39s/it]


KeyboardInterrupt: 

In [91]:
tmp = "- NMS/APM/SMS 등 개발 경험자 우대\n - 스타트업 4년이상 경험자 우대\n - 백엔드 개발 스킬 보유자 우대"
tmp.replace("- ", "").split("\n")

['NMS/APM/SMS 등 개발 경험자 우대', ' 스타트업 4년이상 경험자 우대', ' 백엔드 개발 스킬 보유자 우대']

# 크롤링 결과 및 확인

In [106]:
print(total[0])

key_list = [key for key in total[0].keys()]
print(key_list)

{'id': '165306', 'job': '웹 개발자', 'title': '웹 서버/백엔드(Back-End) 개발자', 'company': '엔드림', 'location': ['서울', '한국'], 'tags': ['연봉업계평균이상', '인원급성장', '퇴사율 6~10%', '51~300명', '설립4~9년', '육아휴직', '자율복장', '워크샵', 'IT, 컨텐츠'], 'qualifications': ['• C# 개발 경험이 있거나, 개발이 가능 하신분 (필수)', '• .NET MVC 개발 경험이 있으신 분', '• MVC 패턴에 대하여 이해하고 계신 분 ', '• HTML5, CSS, javascript 에 대한 이해가 있는 분', '• 외부 Library나 API 사용에 능숙 하신 분'], 'preferred': ['• Java, node.js 개발 경험이 있으신 분', '• 아키텍쳐 설계에 능숙 하신 분', '• HTTP 기반의 대용량 웹 관련 서비스에 대한 경험이 있으신 분', '• 코드 품질 향상을 위한 분석 및 리팩토링 경험이 있으신 분', '• 업무 커뮤니케이션이 능숙 하신 분'], 'benefits': ['■ 오직 엔드리머만이 경험할 수 있는 특별한 복지!', '', 'ㆍ일하기 즐거운 엔드리머', '조직 발전을 위한 2021년 연봉조정대상 전원 연봉 1천만원 파격 인상', '경찰병원역 도보 1분 및 가락시장역 도보 5분 이내에 위치한 사무실', '원거리 근무자의 편의를 위한 오피스텔 임대 지원(2인 1실)', '워라밸(Work-life balance) 향상을 위한 선택적근로시간제 운영', '식권대장을 통한 점심/저녁 식대 제공(일 최대 2만 원)', '임직원 전용 휴게공간 및 커피, 각종 차 제공', '메일 아침을 든든하게 만들어줄 맛 좋은 쿠키와 빵 제공', '사내 호칭을 ‘님’으로 통일해 상호 존중하는 조직문화 지향', '자율복장제도를 통해 캐주얼한 업무 분위기 조성', '', 'ㆍ모두가 건강한 엔드리머', '각종 경조사비 및 경조 화

# 결과 csv 파일로 저장하기

In [101]:
import csv

with open('/Users/jhnam/workspace/playground-data/crawler/wanted_crawling_data_test4.csv', 'w', encoding='UTF-8') as f:
    # csv 파일의 헤더 설정
    header = ['id', 'job', 'title', 'company', 'location', 'tags', 'qualifications', 'preferred', 'benefits', 'skills', 'workplace']

    # 헤더 적용
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    
    # 헤더에 맞게 딕셔너리 값 입력
    for jd in total:
        writer.writerow(jd)

# 전처리 (진행중)

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


exception_list = ['를', '를', '를', '을', '을', '있거나', '있으신', '갖추신', '경험이', '이상의', '에', '에', '에', '에', '에', '이용한', '에', '에', '에', '준하는', '에', '있으신', '에', '에', '에', '중심적인', '에', '에', '이해를', '에', '대한', '및', '가지고', '전반적인', '에']

# 텍스트 정의
contents = "• 6년 이상의 백엔드 개발 경력이 있거나 그에 준하는 실력을 갖추신 분\n• JavaScript / TypeScript 를 이용한 개발에 능숙하신 분\n• 웹 서비스에 대한 전반적인 이해를 갖추신 분\n• RDB(Postgres) 및 ORM에 대한 전반적인 이해를 갖추신 분\n• 도커, 쿠버네티스 등 컨테이너 기반의 서비스에 대한 경험이 있으신 분\n• 1년 이상의 NestJS 실무 프로젝트 개발 경험이 있으신 분\n• 모듈화, 테스트, 유지보수가 용이한 코드에 대한 이해를 갖추고 계신 분\n• 기술 중심이 아닌 유저 중심적인 태도를 가지고 계신 분\n• 배움에 대한 열정을 갖고, 지속적이고 효과적으로 피드백을 주고 받는 분"

# 특수문자 제거
contents = re.sub(r'[^\w\s]', '', contents)

# 단어 토큰화
tokens = word_tokenize(contents)

# 품사 태깅
tagged_words = pos_tag(tokens)

# 명사 추출 (동사 및 불용어 제외)
nouns = [word for word, pos in tagged_words if pos.startswith('N') and not pos.startswith('V') and word.lower() not in exception_list]

# 중복 제거
nouns = list(set(nouns))

# 결과 출력
print(nouns)